# Part 1: Building MCP Servers with Temporal

Learning Objectives:
- Explore the role that MCP plays in AI applications
- Review MCP fundamentals
- Add durability into your MCP tools with Temporal

### Demo 1: Finding Tools Dynamically (Expand for instructor notes or to run on your own)
<!--
Prep:
1. Clone this repository: `https://github.com/temporal-community/durable-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing weather.py.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. Start the Temporal Worker: `uv run run_weather_worker.py` 
3. Open Claude Desktop. 
3. Ask the tool to get weather for city, for example: "What is the weather in Cary, NC?"
4. Emphasize how Claude Desktop discovers that tool dynamically. Allow it to call the `get_weather` tool.
4. Emphasize, but what about reliability?
-->

### Demo #2: Durable Tools with Temporal (Expand for instructor notes or to run on your own).
<!--
Note that this demo is optional, because if students did the first workshop (Agentic Loop and Temporal), students should understand how Temporal maintains durability. However, feel free to run this demo if you have extra time or you want to emphasize the point.
Prep:
1. Clone this repository: `https://github.com/temporal-community/durable-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing server.py.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. You'll notice this repository includes a `pf.rules` file that has URLs for the news weather API. We will block those to imitate a network outage for that API.
3. Set the rules with `sudo pfctl -f pf.rules`.
4. Enable the firewall with `sudo pfctl -e`.
5. Start the Temporal Worker: `uv run run_weather_worker.py` 
6. Restart Claude Desktop. 
7. Ask the tool to get weather for city, for example: "What is the weather in Cary, NC?"
8. Go on the Web UI and point out that the `make_nws_request` Activity is retrying.
9. Disable the firewall with `sudo pfctl -d`.
10. Watch the Workflow Execution complete successfully.
-->

### Demo #3: Exiting Out Claude Desktop while Tool is Executing (Expand for instructor notes or to run on your own)
<!--
Prep:
1. Clone this repository: `https://github.com/temporal-community/durable-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing server.py.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo:
1. Run your temporal server: `temporal server start-dev`. 
2. Start the Temporal Worker: `uv run run_weather_worker.py` 
3. Restart Claude Code, and ask the tool to get weather for city, for example: "What is the weather in Cary, NC?"
4. Allow the tool to be used, then exit out Claude Code.
5. Go on the Web UI, and demonstrate that the Workflow is still running. 
6. Emphasize that with Temporal, the tool keeps going. The Workflow runs independently of the MCP server process.
-->

### Make Sure Your Temporal Web UI is Running

1. Run `temporal server start-dev` in your terminal.
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

### Activities

An Activity is code that is prone to failure, non-deterministic, making external calls etc.

In [ ]:
# Step 1: Make the code an Activity. Look at the cell below for the solution.
# Step 2: Now run the code to load it into the program

from typing import Any
from temporalio import activity
import httpx

USER_AGENT = "weather-app/1.0"

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:

        response = await client.get(url, headers=headers, timeout=5.0)
        response.raise_for_status()
        return response.json()

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Workflow

The Workflow orchestrates Activities and maintains state durably.

In [ ]:
# Step 1: Make the Workflow call the `make_nws_request` Activity for both `points_data` and `forecast_data`. 
# Step 2: Set the Schedule-to-Close to be 40 seconds in each Activity call.
# Step 3: Now run the code to load it into the program

from temporalio import workflow
from datetime import timedelta
from temporalio.common import RetryPolicy

retry_policy = RetryPolicy(
    maximum_attempts=0,  # Infinite retries
    initial_interval=timedelta(seconds=2),
    maximum_interval=timedelta(minutes=1),
    backoff_coefficient=1.0,
)

# Constants
NWS_API_BASE = "https://api.weather.gov"

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GetForecast:
    @workflow.run
    async def get_forecast(self, latitude: float, longitude: float) -> str:
        """Get weather forecast for a location.

        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
        """
        # First get the forecast grid endpoint
        points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
        points_data = await workflow.execute_activity(
            # TODO: Execute `make_nws_request` Activity,
            points_url,
            # TODO: Set Schedule-to-Close Timeout to be 40 seconds
            retry_policy=retry_policy,
        )

        if not points_data:
            return "Unable to fetch forecast data for this location."

        await workflow.sleep(10)

        # Get the forecast URL from the points response
        forecast_url = points_data["properties"]["forecast"]
        forecast_data = await workflow.execute_activity(
            # TODO: Execute `make_nws_request` Activity,
            forecast_url,
            # TODO: Set Schedule-to-Close Timeout to be 40 seconds
            retry_policy=retry_policy,
        )
        if not forecast_data:
            return "Unable to fetch detailed forecast."

        # Format the periods into a readable forecast
        periods = forecast_data["properties"]["periods"]
        forecasts = []
        for period in periods[:5]:  # Only show next 5 periods
            forecast = f"""
    {period['name']}:
    Temperature: {period['temperature']}°{period['temperatureUnit']}
    Wind: {period['windSpeed']} {period['windDirection']}
    Forecast: {period['detailedForecast']}
    """
            forecasts.append(forecast)

        return "\n---\n".join(forecasts)

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "workflow_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))


### Create Your MCP Tool

Create MCP tool which calls the Workflow. This MCP tool is now durable!

In [ ]:
# Step 1: Call the `GetForecast` Workflow as a string.
# Step 2: Now run the code to load it into the program

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with a protocol
# Therefore, we also have this code in a separate Python file that can be run
# as a standalone MCP server (mcp_servers/weather.py).

from temporalio.client import Client
from fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("weather")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    client = await get_temporal_client()
    handle = await client.start_workflow(
        workflow= # TODO: Call the `GetForecast` Workflow. It should be a string.
        args=[latitude, longitude],
        id=f"forecast-{latitude}-{longitude}",
        task_queue="weather-task-queue",
    )
    return await handle.result()

    if __name__ == "__main__":
        # Initialize and run the server
        mcp.run(transport="sse", host="0.0.0.0", port=5125)

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "mcp_server_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))


### Run Your Worker

Temporal Workflows and Activities are run in Workers

In [ ]:
# Run this codeblock

import asyncio
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    # Connect to Temporal server (change address if using Temporal Cloud)
    client = await Client.connect("localhost:7233")

    worker = Worker(
        client,
        task_queue="weather-task-queue",
        workflows=[GetForecast],
        activities=[make_nws_request],
    )
    print("Worker started. Listening for workflows...")
    await worker.run()


In [ ]:
# Due to the limitation of Jupyter Notebooks and Google Collab, this is how
# you must start the worker in a Notebook environment
worker = asyncio.create_task(run_worker())

# If you are running this code in a typical Python environment, you can start
# the Worker by just calling `asyncio.run`
# if __name__ == "__main__":
#    asyncio.run(run_worker())

### Configure Your MCP Client
#### Take 15 minutes to do the following steps.

1. Open a new terminal window, seperate from the one you are running the Temporal server with, and start your MCP Server with: `uv run mcp_servers/weather.py`. Leave this terminal running.
2. We will start our Streamlit Interface in the next step. Open a third terminal window.
    - _Streamlit is a Python framework used for building interactive web applications. We have built one for you that uses an MCP Client._
3. Load your environment variables and start Streamlit:
    - a. Run `chmod +x setup-env.sh` to make this script executable
    - b. Run `source setup-env.sh` to load your environment variables. This makes your API key available to the Streamlit application running locally in your Codespace.
    - c. Run `uv run streamlit run mcp_client_interface.py`. Leave this terminal window and the new window that pops up running for the rest of the workshop.
4. In the new window that pops up, click the "Load MCP Tools" button. You should see the `get_forecast` tool available.
5. In the chat interface, try asking something like: "What's the weather forecast for San Francisco, CA?"
6. OpenAI will decide to use the `get_forecast` tool, and your MCP Client will call this tool from your MCP server.
7. Open your Web UI to see your Workflows executing in real-time!
    - To open your Temporal Web UI, in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon.
    - What were the inputs and outputs of both `make_nws_request` Activities?
    - What was the final output of the Workflow Execution?

In [ ]:
# Kill any worker to prepare for the next demo.
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

### Simulate a Bug in Error Handling!

In this next section, we'll produce a bug where retries don't work as expected. You'll observe the failure, see retries not working as expected, and then identify and fix the error.

In [ ]:
# Buggy Activity Code

# Step 1: This Activity has a bug that prevents retries from working!
# Can you spot it before running the code? Run the cell below to see the answer.
# Step 2: Run this codeblock.

from typing import Any
from temporalio import activity
from temporalio.exceptions import ApplicationError
import httpx

USER_AGENT = "weather-app/1.0"

@activity.defn
async def make_nws_request_buggy(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API - but with a bug!"""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    
    try:
        # Simulate a timeout error
        raise ApplicationError(
            "Simulated timeout: Weather service temporarily unavailable"
        )
        
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers, timeout=5.0)
            response.raise_for_status()
            return response.json()
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [ ]:
# Optional: Run this cell to load and display what the bug is
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "buggy_activity_solution.md"

response = solution_file.read_text()

print("Solution loaded:")
display(response)

In [ ]:
# Run this Workflow that calls the buggy Activity
from temporalio import workflow
from datetime import timedelta
from temporalio.common import RetryPolicy

NWS_API_BASE = "https://api.weather.gov"

retry_policy = RetryPolicy(
    maximum_attempts=0,  # Infinite retries
    initial_interval=timedelta(seconds=2),
    maximum_interval=timedelta(minutes=1),
    backoff_coefficient=1.0,
)

@workflow.defn(sandboxed=False)
class GetForecast:
    @workflow.run
    async def get_forecast(self, latitude: float, longitude: float) -> str:
        """Get weather forecast - buggy version."""
        points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
        
        points_data = await workflow.execute_activity(
            make_nws_request_buggy,
            points_url,
            start_to_close_timeout=timedelta(seconds=40),
            retry_policy=retry_policy
        )

        if not points_data:
            return "Unable to fetch forecast data - but no retries happened!"

        forecast_url = points_data["properties"]["forecast"]
        forecast_data = await workflow.execute_activity(
            make_nws_request_buggy,
            forecast_url,
            start_to_close_timeout=timedelta(seconds=40),
            retry_policy=retry_policy
        )
        
        if not forecast_data:
            return "Unable to fetch detailed forecast - but no retries happened!"

        periods = forecast_data["properties"]["periods"]
        forecasts = []
        for period in periods[:5]:
            forecast = f"""
    {period['name']}:
    Temperature: {period['temperature']}°{period['temperatureUnit']}
    Wind: {period['windSpeed']} {period['windDirection']}
    Forecast: {period['detailedForecast']}
    """
            forecasts.append(forecast)

        return "\n---\n".join(forecasts)

In [ ]:
# Define a Worker that registers the buggy Activity and Workflow
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    client = await Client.connect("localhost:7233")
    
    worker = Worker(
        client,
        task_queue="weather-task-queue",
        workflows=[GetForecast],
        activities=[make_nws_request_buggy],
    )
    print("Worker started. Listening for workflows...")
    await worker.run()

In [ ]:
# Start the Worker
import asyncio

worker = asyncio.create_task(run_worker())

In [ ]:
# Run this codeblock to test the workflow

from temporalio.client import Client
import uuid

async def test_buggy_workflow():
    client = await Client.connect("localhost:7233")
    
    handle = await client.start_workflow(
        GetForecast,
        args=[37.7749, -122.4194],
        id=f"buggy-forecast-test-{uuid.uuid4()}",
        task_queue="weather-task-queue",
    )
    
    return handle

await test_buggy_workflow()

### Observing Your Buggy Workflow

Observe your Web UI. We probably expect that the Activity would retry on failure. But what we actually see is that the Workflow has completed succsessfully! 

However, if you click on the `ActivityTaskCompleted` Event, we can see we didn't get the forecast data. We seemed to have run into an error, but no retries happened, and the Workflow still completed successfully! What happened here?

In [ ]:
# Stop the Worker before we fix it

x = worker.cancel()

In [ ]:
# Now let's fix the bug by removing the try-except block and the simulated error
# Step 1: Remove the `try` and `except` statements
# By doing this, we let exceptions propagate to Temporal
# So if an exception occurs, Temporal will see it and trigger retries!
# Step 2: Fix your indentations, and run this codeblock

from typing import Any
import httpx
from temporalio import activity
from temporalio.exceptions import ApplicationError

USER_AGENT = "weather-app/1.0"

@activity.defn
async def make_nws_request_buggy(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API"""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    
    try: # TODO: Remove the try statement
        raise ApplicationError(
            "Simulated timeout: Weather service temporarily unavailable"
        )
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers, timeout=5.0)
            response.raise_for_status()
            return response.json()
    except Exception as e: # TODO: Remove everything in the except statement
        print(f"Error occurred: {e}")
        return None

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "fixed_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Run this codeblock. Register a Worker with the "fixed" code

from temporalio.client import Client
from temporalio.worker import Worker

async def run_fixed_worker():
    client = await Client.connect("localhost:7233")
    
    worker = Worker(
        client,
        task_queue="weather-task-queue",
        workflows=[GetForecast],
        activities=[make_nws_request_buggy],
    )
    print("Fixed worker started. Listening for workflows...")
    await worker.run()

In [ ]:
# Start a Worker with the fixed code. Run this codeblock.
import asyncio 

worker = asyncio.create_task(run_worker())

In [ ]:
# Test the "fixed" code. Run this codeblock.

from temporalio.client import Client
import uuid

async def test_fixed_workflow():
    client = await Client.connect("localhost:7233")
    
    latitude, longitude = 37.7749, -122.4194
    workflow_id = f"fixed-forecast-test-{uuid.uuid4()}"
    
    handle = await client.start_workflow(
        GetForecast.get_forecast,
        args=[latitude, longitude],
        id=workflow_id,
        task_queue="weather-task-queue",
    )
    
await test_fixed_workflow()

### Remove the Simulated Failure

Now let's "fix" our simulated failure by removing the error. In a real scenario, this could be:

- A database coming back online
- An API endpoint being fixed
- A network issue being resolved

Fix the code by removing or commenting out the error.

In [ ]:
# Step 1: Remove the ApplicationError
# Step 2: Run the codeblock from typing import Any
from typing import Any
import httpx
from temporalio import activity
from temporalio.exceptions import ApplicationError

USER_AGENT = "weather-app/1.0"

@activity.defn
async def make_nws_request_buggy(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API"""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    raise ApplicationError( # Remove the error by commenting it out or deleting it
        "Simulated timeout: Weather service temporarily unavailable"
    )
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers, timeout=5.0)
        response.raise_for_status()
        return response.json()

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "01_MCP_Temporal_Intro_Solution" / "removed_exception_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Restart the Worker with Fixed Code

Now restart the Worker so it picks up the fixed Activity code:

In [ ]:
# Kill the old worker and start the new one
import asyncio

x = worker.cancel()
worker = asyncio.create_task(run_worker())
print("New worker started!")

In [ ]:
# Run this codeblock to test the fixed code 

from temporalio.client import Client
import uuid

async def test_fixed_workflow():
    client = await Client.connect("localhost:7233")
    
    # Using valid coordinates for San Francisco
    latitude, longitude = 37.7749, -122.4194
    workflow_id = f"fixed-forecast-test-{uuid.uuid4()}"
    
    handle = await client.start_workflow(
        GetForecast.get_forecast,
        args=[latitude, longitude],
        id=workflow_id,
        task_queue="weather-task-queue",
    )
    
await test_fixed_workflow()

### Observe Your Web UI

Your Workflow Execution should now complete successfully.

In [ ]:
# Kill any worker to prepare for the exercise.
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

---
## Exercise 1 - Making Tools Durable

* In this exercise, you will:
  * Build durability and persistence to your MCP tools with Temporal Workflows
  * Test the integration between Claude Desktop, MCP servers, and Temporal workflows
* Go to the **exercises** Directory and open the `01_Making_MCP_Tools_Durable` folder. Then, open the **practice** directory.
* Open the `README.md` and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers

### What's Next?

This workshop introduced you to the **importance of durable MCP tools** and insights in exception handling. Further your learning with these resources:

### Resources

- [Free Temporal Error Handling Course](https://learn.temporal.io/courses/errstrat/python/)
- [Weather Demo GitHub Repository](https://github.com/temporal-community/durable-mcp)